In [1]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14, 6)
import numpy as np
import librosa, librosa.display, IPython.display as ipd
from tqdm import tqdm
import glob
import pandas as pd
import h5py
from sklearn import preprocessing
import pickle
np.random.seed(1515)

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config) 
    
import keras 
from keras.backend.tensorflow_backend import set_session
set_session(sess)  # set this TensorFlow session as the default session for Keras

from keras.layers import Input, Conv2D, MaxPooling2D, LSTM, GRU, Bidirectional
from keras.layers import Dense, Flatten
from keras.layers.core import Dropout
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [3]:
#SET PATHS
dataspace = '/vol/vssp/datasets/audio01/UrbanSound8K/audio/'
projectspace = '/vol/vssp/AcousticEventsDetection/DLGdansk/UrbanSound/'

metadatafile = '/vol/vssp/datasets/audio01/UrbanSound8K/metadata/UrbanSound8K.csv'
metadata = pd.read_csv(metadatafile)

hdf5_path = os.path.join(projectspace,'dataset.hdf5') 
modelfolder = os.path.join(projectspace,'models')
scalerpath = os.path.join(projectspace,'scaler.pkl')

In [4]:
# LOAD DATA
hf = h5py.File(hdf5_path, 'r')
X_train = np.array(hf.get('X_train') )
y_train = np.array(hf.get('y_train'))
X_val =  np.array(hf.get('X_val'))
y_val= np.array(hf.get('y_val'))
X_test = np.array(hf.get('X_test'))
y_test = np.array(hf.get('y_test'))

n_features= hf.get('n_features').value
n_frames=hf.get('max_length_samp').value
label_list= hf.get('label_list').value

In [5]:
# SCALE THE DATA
scaler=pickle.load(open(scalerpath, 'rb'))

X_train_scaled = [scaler.transform(x.T) for x in X_train]
X_val_scaled = [scaler.transform(x.T) for x in X_val]
X_test_scaled = [scaler.transform(x.T) for x in X_test]


In [6]:
len(y_val)

816

In [7]:
def build_rnn(n_frames=n_frames, n_features=n_features,
                     n_classes=10):

    # INPUT
    x = Input(shape=(n_frames, n_features), dtype='float32')
  
    y= LSTM(120, return_sequences = True)(x)
    #y= Dropout(0.5)(y)  
    
    y= LSTM(120)(y)
    #y= Dropout(0.5)(y) 

    y = Dense(n_classes, activation='sigmoid')(y)

    m = Model(inputs=x, outputs=y)
    return m

In [8]:
m = build_rnn(n_features=40,)
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 173, 40)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 173, 120)          77280     
_________________________________________________________________
lstm_2 (LSTM)                (None, 120)               115680    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1210      
Total params: 194,170
Trainable params: 194,170
Non-trainable params: 0
_________________________________________________________________


In [9]:
m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_checkpoint = ModelCheckpoint(
    os.path.join(modelfolder, 'rnn_epoch_{epoch:03d}_val_loss_{val_loss:.4f}.hdf5'),
    monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping, model_checkpoint]

history = m.fit(x=np.array(X_train_scaled), y=y_train, batch_size=130,
                    epochs=200, verbose=True,
                    validation_data=(np.array(X_val_scaled), y_val), shuffle=True)

Train on 7079 samples, validate on 816 samples
Epoch 1/200
7079/7079 [==============================] - 28s 4ms/step - loss: 1.8441 - acc: 0.3009 - val_loss: 1.8088 - val_acc: 0.3064
Epoch 2/200
7079/7079 [==============================] - 28s 4ms/step - loss: 1.5702 - acc: 0.4037 - val_loss: 1.6956 - val_acc: 0.3456
Epoch 3/200
7079/7079 [==============================] - 28s 4ms/step - loss: 1.5223 - acc: 0.4389 - val_loss: 1.5602 - val_acc: 0.4093
Epoch 4/200
7079/7079 [==============================] - 28s 4ms/step - loss: 1.3949 - acc: 0.4940 - val_loss: 1.4376 - val_acc: 0.4473
Epoch 5/200
7079/7079 [==============================] - 28s 4ms/step - loss: 1.3831 - acc: 0.5005 - val_loss: 1.6010 - val_acc: 0.4142
Epoch 6/200
7079/7079 [==============================] - 27s 4ms/step - loss: 1.3234 - acc: 0.5227 - val_loss: 1.4830 - val_acc: 0.4890
Epoch 7/200
7079/7079 [==============================] - 27s 4ms/step - loss: 1.2526 - acc: 0.5550 - val_loss: 1.5531 - val_acc: 0.4988
E

In [ ]:
prediction = m.predict(X_test, batch_size=130, verbose=1)
y_predict=np.array([ np.argmax(p) for p in prediction])
y_test_label = np.array([np.argmax(y) for y in y_test])

In [ ]:
from sklearn.metrics import confusion_matrix

acc = np.sum(y_test_label==y_predict) / float(len(y_test_label))
print('Accuracy: {:.2f}'.format(acc))

cm = confusion_matrix(y_test_label, y_predict )
sns.heatmap(cm, annot=True, annot_kws={"size": 14}, fmt='g', linewidths=.5, yticklabels=label_list,xticklabels=label_list)